<a href="https://colab.research.google.com/github/dinesh-umkc/kdm/blob/main/ICP_11_BERTopic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Objective

* Explore topic modeling with Bert
* Compare BERT topic model with LDA
* Use reddit dataset for topic modeling
* Visualize topic


# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [9]:
%%capture
!pip install bertopic
!pip install --upgrade joblib==1.1.0

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data
Use Reddit Data

In [2]:
# !pip install praw
!pip install asyncpraw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 195 kB 5.3 MB/s 


In [3]:
# import praw
import asyncpraw as praw
from asyncpraw.models import MoreComments
reddit = praw.Reddit(
    user_agent="Comment Extraction (by u/Bitter_Researcher859)",
    client_id="5YUVEJjNjYgNfwrR0Jle_Q",
    client_secret="y1Yw5hcMa874BIzhbG5DSFnE9f_ncg",
    redirect_uri='http://127.0.0.1:65010/'
                                 'authorize_callback'
    # username="USERNAME",
    # password="PASSWORD"
)

In [4]:
import time
from tqdm.notebook import tqdm
topic_library = {
    'War' : [ 'https://www.reddit.com/r/worldnews/comments/z4b7cl/nato_vows_to_aid_ukraine_for_as_long_as_it_takes/'
                  ],
    'Other': [
        'https://www.reddit.com/r/Damnthatsinteresting/comments/z47fk5/the_statue_of_unity_in_india_the_tallest_statue/', 
        ]
}
for topic, urls in tqdm(topic_library.items()):
  for url in tqdm(urls):
    submission = await reddit.submission(url=url)
    submission.comments.replace_more(limit=0) # flatten tree
    all_comments = []
    comments = submission.comments # all comments
    # comments = submission.comments.list() # all comments
    for top_level_comment in submission.comments:
      if isinstance(top_level_comment, MoreComments):
          continue
      all_comments.append(top_level_comment.body)
      with open('{}-reddit_author.txt'.format(topic), 'a') as f:
          f.write(submission.selftext + '\n')
      with open('{}-reddit_comments.txt'.format(topic), 'a') as f:
        for i, comment in enumerate(comments):
          f.write(comment.body + '\n')
    # time.sleep(1.5)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: coroutine 'CommentForest.replace_more' was never awaited
  app.launch_new_instance()


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
#Gdrive access
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
import shutil
for topic, _ in tqdm(topic_library.items()):
  shutil.copy2('{}-reddit_author.txt'.format(topic), '/content/gdrive/MyDrive/Projects/Data/')
  shutil.copy2('{}-reddit_comments.txt'.format(topic), '/content/gdrive/MyDrive/Projects/Data/')

  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
import pandas as pd
reddit_df = pd.DataFrame(columns = ['text'])
for topic, _ in tqdm(topic_library.items()):
  with open('{}-reddit_comments.txt'.format(topic)) as f:
    lines = f.readlines()
    reddit_sub_df = pd.DataFrame(lines, columns = ['text'])
    reddit_sub_df = reddit_sub_df.sample(3000)
    reddit_df= reddit_df.append(reddit_sub_df).reset_index(drop=True)
# lines

  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
reddit_df

,text
0,Let the bodies pile up!\n
1,NATO is not helping at all. It is using Ukrain...
2,[deleted]\n
3,\n
4,Putin go jump out a window\n
...,...
5995,"Yeah, bonjour France I just thought you'd be i..."
5996,"How is that even possible, who built it, how??\n"
5997,"Yeah, but does it lie about asking for your ti..."
5998,\n


# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


In [12]:
!pip install --upgrade joblib==1.1.0
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
topics, probs = topic_model.fit_transform(reddit_df['text'])

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/188 [00:00<?, ?it/s]

2022-11-25 16:38:37,734 - BERTopic - Transformed documents to Embeddings
2022-11-25 16:39:37,746 - BERTopic - Reduced dimensionality
2022-11-25 16:39:46,168 - BERTopic - Clustered reduced embeddings


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [14]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,353,-1_hava_white_purpose_pillars
1,0,83,0____
2,1,49,1_deleted___
3,2,43,2_ukrainians_disposable_made_fighting
4,3,43,3_questions_face_sanctions_recover


In [15]:
freq


,Topic,Count,Name
0,-1,353,-1_hava_white_purpose_pillars
1,0,83,0____
2,1,49,1_deleted___
3,2,43,2_ukrainians_disposable_made_fighting
4,3,43,3_questions_face_sanctions_recover
...,...,...,...
185,184,11,184____
186,185,11,185____
187,186,11,186____
188,187,10,187____


In [16]:
len(freq)

190

-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [17]:
topic_model.get_topic(5)  # Select the most frequent topic

[('im', 0.11523819775373398),
 ('happy', 0.10792841711577485),
 ('accused', 0.08726277310268314),
 ('eu', 0.08726277310268314),
 ('profiting', 0.08726277310268314),
 ('believe', 0.08726277310268314),
 ('nevertheless', 0.08726277310268314),
 ('understand', 0.08726277310268314),
 ('dont', 0.08154970534317296),
 ('sorry', 0.06766488766670581)]

**NOTE**: BERTopic is stocastic which means that the topics might differ across runs. This is mostly due to the stocastisc nature of UMAP.

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [18]:
topic_model.topics_[:10]

[14, 45, 140, 151, 52, 158, 17, 35, 3, 165]

In [19]:
len(topic_model.topics_)

6000

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through a hundred topics to get a good 
understanding of the topics that were extracted. However, that takes quite some time and lacks a global representation. 
Instead, we can quantitatively visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [20]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [21]:
probs.shape

(6000, 189)

In [23]:
import random
random_document_id = random.randint(0, len(probs))
dataf=reddit_df['text']
print(dataf[random_document_id])
print('Topic: {}'.format(topic_model.topics_[random_document_id]))

# topic_model.visualize_distribution(probs[random_document_id], min_probability=0.015)
topic_model.visualize_distribution(probs[random_document_id], min_probability=0.003)

Obliterating it’s fascist forces is essential to securing Europe.

Topic: 140


## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [24]:
topic_model.visualize_hierarchy(top_n_topics=4)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [25]:
topic_model.visualize_barchart(top_n_topics=5)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.


In [26]:
topic_model.visualize_heatmap(top_n_topics = 20, width=1000, height=1000)

You can set `n_clusters` in `visualize_heatmap` to order the topics by their similarity. This will result in blocks being formed in the heatmap indicating which clusters of topics are similar to each other. This step is very much recommended as it will make reading the heatmap easier.

In [27]:
topic_model.visualize_heatmap(n_clusters = 3, top_n_topics = 20, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [28]:
topic_model.visualize_term_rank(log_scale=True)

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [29]:
topic_model.update_topics(dataf, n_gram_range=(2, 3))

In [30]:
topic_model.get_topic(2)   # We select topic that we viewed before

[('money to be', 0.12912472974480832),
 ('made fighting', 0.12912472974480832),
 ('so much money', 0.12912472974480832),
 ('so much', 0.12912472974480832),
 ('russians with disposable', 0.12912472974480832),
 ('russians with', 0.12912472974480832),
 ('much money to', 0.12912472974480832),
 ('much money', 0.12912472974480832),
 ('with disposable ukrainians', 0.12912472974480832),
 ('money to', 0.12912472974480832)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [31]:
topic_model.reduce_topics(dataf, nr_topics=60)

2022-11-25 16:45:32,100 - BERTopic - Reduced number of topics from 190 to 61


In [32]:
# Access the newly updated topics with:
print(topic_model.topics_)

[54, 18, -1, -1, 1, -1, 17, 3, 39, -1, -1, -1, -1, 8, -1, 19, 4, 9, 4, -1, 42, 59, -1, 6, -1, 50, 58, 6, -1, 34, -1, 9, 18, -1, -1, 52, 4, 0, 17, 48, 58, -1, 16, 49, -1, 15, 35, 58, 36, 47, 39, 44, 10, 51, 6, 58, 15, 41, 53, 52, 3, -1, 54, 43, 18, 17, 15, 34, -1, -1, 55, 43, -1, 1, 35, -1, 39, 3, 58, 44, 36, 10, 1, 51, -1, 18, -1, 6, 15, 3, -1, 43, 35, 38, -1, 19, 12, -1, 3, -1, -1, 50, -1, 52, -1, 38, 12, 47, 53, -1, 52, 6, 10, -1, 35, 59, 50, 4, 10, 44, 13, 40, -1, 16, 8, 48, 11, 33, 40, 13, 54, 53, 18, 0, -1, 11, -1, 17, 14, -1, -1, 42, 48, 46, 50, 56, 0, -1, -1, -1, 4, -1, 18, 51, 43, -1, 48, -1, 44, 54, 57, -1, 44, 52, 38, 43, -1, 13, -1, 9, 33, 14, 58, 18, 4, 38, -1, -1, -1, 48, 13, 3, 57, -1, 54, 44, 47, -1, 9, -1, 11, -1, -1, 45, 54, 0, 37, 45, -1, 18, -1, 11, 37, -1, 11, -1, 48, 13, 15, 57, 13, -1, 4, -1, 3, -1, 39, -1, 13, 46, 59, 34, 45, -1, 38, 4, -1, 52, -1, 8, -1, 13, 59, 47, 3, 59, 10, 44, 17, 1, -1, 16, -1, -1, 58, -1, 37, 17, 51, 42, -1, -1, 34, -1, 6, 9, 6, -1, 3, 50,

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [33]:
similar_topics, similarity = topic_model.find_topics("war", top_n=5); similar_topics

[57, 53, 38, 18, 6]

In [36]:
topic_model.get_topic(53)

[('fighting it', 0.13214363546598298),
 ('them the', 0.13214363546598298),
 ('wont give', 0.13214363546598298),
 ('win the warjust', 0.13214363546598298),
 ('win the', 0.13214363546598298),
 ('weapon systems they', 0.13214363546598298),
 ('weapon systems', 0.13214363546598298),
 ('warjust keep fighting', 0.13214363546598298),
 ('warjust keep', 0.13214363546598298),
 ('to win the', 0.13214363546598298)]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. 

In [37]:
# Save model
topic_model.save("/content/gdrive/MyDrive/Projects/Data/bert_model")	

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [38]:
# Load model
my_model = BERTopic.load("/content/gdrive/MyDrive/Projects/Data/bert_model")	

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [39]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens") #Trained on NLI and STSB datasets

Or select a SentenceTransformer model with your own parameters:


In [40]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]